# RAG Pipeline: Evaluation and Analysis

This notebook focuses on running the complete Retrieval-Augmented Generation (RAG) pipeline on a set of representative questions. We will analyze how the system retrieves relevant complaints and generates answers using the LLM.

### Pipeline Components:
1. **Vector Store**: FAISS index containing complaint embeddings.
2. **Retriever**: Similarity search mechanism to find top-K relevant documents.
3. **LLM**: Instruction-tuned model (FLAN-T5) to generate answers based on context.
4. **Prompt Engineering**: Structured templates to guide the LLM's response.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import IPython
from IPython.display import display, Markdown

# allow imports from project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src import config, vectorstore, llm
from src.rag_pipline import RAGPipeline

config.setup_hf_cache()
print("✓ Environment setup complete")

## 1. Initialize Production Pipeline

We initialize the `RAGPipeline` class, which encapsulates the loading of the vector store and the LLM engine.

In [ ]:
pipeline = RAGPipeline()
if pipeline.initialize():
    print("✓ RAG Pipeline initialized and ready!")
else:
    print("❌ Pipeline initialization failed. Check your vector store and model paths.")

## 2. Representative Evaluation Questions

Here is a curated list of questions that represent common consumer concerns. We will use these to evaluate the system's performance.

In [ ]:
eval_questions = [
    "What are common issues consumers face with credit card interest rates?",
    "How do customers describe problems with mortgage loan modifications?",
    "What are the most frequent complaints regarding unauthorized transactions on checking accounts?",
    "What reasons do consumers give for disputes regarding credit reporting inaccuracies?",
    "What are the common themes in complaints against debt collection agencies' practices?",
    "How do consumers react to unexpected fees on prepaid cards?",
    "What are the typical complaints about vehicle loan servicing?",
    "What issues do consumers report when trying to close their savings accounts?"
]

print(f"✓ Defined {len(eval_questions)} representative questions.")

## 3. Running RAG Pipeline and Results Analysis

For each question, we run the full RAG cycle:
1. **Retrieve**: Find top 5 relevant complaint chunks.
2. **Augment**: Format the chunks into a context string for the prompt.
3. **Generate**: Use FLAN-T5 to generate a professional answer based *only* on the context.

In [ ]:
all_results = []

for i, query in enumerate(eval_questions, 1):
    display(Markdown(f"### Question {i}: {query}"))
    
    # Run the pipeline
    result = pipeline.run(query, k=5)
    
    # Display the answer
    display(Markdown(f"**Generated Answer:** {result['answer']}"))
    
    # Display source metadata summary
    sources = result['source_documents']
    display(Markdown(f"*Retrieved {len(sources)} sources from products: {set([doc.metadata.get('product', 'N/A') for doc in sources])}*"))
    
    # Show a snippet of the most relevant source
    top_source = sources[0]
    display(Markdown(f"**Top Source Snippet (Complaint {top_source.metadata.get('complaint_id')}):**  \n*{top_source.page_content[:250]}...*"))
    
    display(Markdown("---"))
    
    all_results.append({
        "question": query,
        "answer": result['answer'],
        "sources": sources
    })

## 4. Pipeline Evaluation Table

We consolidate the results into a structured evaluation table. This can be exported to your report. 

> **Note:** The `Quality Score` and `Comments/Analysis` columns are intended for manual review to ensure the AI's answers are grounded and accurate.

In [ ]:
evaluation_data = []

for res in all_results:
    # Get 1-2 source IDs
    source_ids = [doc.metadata.get('complaint_id', 'N/A') for doc in res['sources'][:2]]
    
    evaluation_data.append({
        "Question": res['question'],
        "Generated Answer": res['answer'],
        "Retrieved Sources": ", ".join(map(str, source_ids)),
        "Quality Score (1-5)": "",  # Placeholder for manual review
        "Comments/Analysis": ""      # Placeholder for manual review
    })

df_eval = pd.DataFrame(evaluation_data)

# Set column display for better readability in the notebook
pd.set_option('display.max_colwidth', 300)
display(df_eval)

# Optional: Save to CSV for the report
# df_eval.to_csv("rag_evaluation_report.csv", index=False)